In [1]:
%%time
import cx_Oracle
import numpy as np
import pandas as pd

#고객별 2014년 2015년 총구매액 및 평균구매액 출력
conn=cx_Oracle.connect('c##hr/hr@localhost:1521/xe')
cursor = conn.cursor()

Wall time: 2.38 s


In [3]:
cursor.execute('SELECT * FROM demo')
x = cursor.fetchall()

cursor.execute("select column_name from user_tab_columns where table_name= 'DEMO'")
y = cursor.fetchall()
colst=[]
for i in y:
    for j in i:
        colst.append(j)


df1 = pd.DataFrame(x, columns=colst)
df1

,고객번호,성별,연령대,거주지역
0,00501,F,60세이상,460
1,00502,F,60세이상,060
2,00503,F,60세이상,055
3,00504,F,60세이상,049
4,00505,F,60세이상,055
...,...,...,...,...
19378,19261,F,60세이상,072
19379,19262,F,20세~24세,052
19380,19263,F,60세이상,460
19381,19264,F,45세~49세,100


In [4]:
cursor.execute('select 고객번호, count(이용년월) from comp group by 고객번호 order by 고객번호')
x = cursor.fetchall()

df2 = pd.DataFrame(x, columns=['고객번호','경쟁사이용횟수'])
df2

,고객번호,경쟁사이용횟수
0,00001,15
1,00002,8
2,00004,3
3,00007,1
4,00008,3
...,...,...
5876,19370,24
5877,19372,4
5878,19375,6
5879,19376,1


In [5]:
cursor.execute('select 고객번호, 멤버십명 from mem order by 고객번호');
x = cursor.fetchall()

df3 = pd.DataFrame(x, columns=['고객번호','멤버쉽명'])
df3

,고객번호,멤버쉽명
0,00011,하이마트
1,00021,하이마트
2,00037,하이마트
3,00043,하이마트
4,00044,하이마트
...,...,...
7451,19377,롭스
7452,19379,롭스
7453,19380,롭스
7454,19381,다둥이


In [6]:
df=pd.merge(df1,df2, on='고객번호',how='left')
df=pd.merge(df,df3, on='고객번호',how='left')
df=df.sort_values(by='고객번호')
df=df.reset_index().drop(['index'], axis=1)
df

,고객번호,성별,연령대,거주지역,경쟁사이용횟수,멤버쉽명
0,00001,M,60세이상,060,15.0,NaN
1,00002,M,60세이상,100,8.0,NaN
2,00003,M,60세이상,033,NaN,NaN
3,00004,F,60세이상,016,3.0,NaN
4,00005,M,60세이상,100,NaN,NaN
...,...,...,...,...,...,...
20336,19379,F,20세~24세,069,NaN,롭스
20337,19380,F,20세~24세,039,NaN,롭스
20338,19381,F,40세~44세,460,NaN,다둥이
20339,19382,F,45세~49세,460,NaN,NaN


In [94]:
#멤버쉽이 2개 이상 있는 고객
dup = df.duplicated(['고객번호'], keep='first')
dup

sum=0
for i in dup:
    if i==True:
        sum+=1
print(sum,'명')

958 명


In [92]:
#2가지 보유 958-61
id=0
m=0
m3=0
t=0
g=0
h=0

sum=0
for i,j in zip(df['고객번호'],df['멤버쉽명']):
    if i==m:
#         print(i,'2장')
        sum+=1
    m=i
print(sum-61,'명')

897 명


In [89]:
#3가지 보유 61-1명
sum=0
for i,j in zip(df['고객번호'],df['멤버쉽명']):
    if i==m:
#         print(i,'2장')
        if t==m:
            sum+=1
        t=m
    m=i
print(sum-1,'명')

60 명


In [100]:
#4가지 보유
sum=0

for i,j in zip(df['고객번호'],df['멤버쉽명']):
    if i==m:
#         print(i,'2장')
        if t==m:
#             print(i,'3장')
            if g==t:
                print('고객번호>',i,' 4장')
            g=t
        t=m
    m=i


하이마트
고객번호> 13894  4장


In [102]:
df[df['고객번호'] == '13894']

,고객번호,성별,연령대,거주지역,경쟁사이용횟수,멤버쉽명
14556,13894,F,30세~34세,460,NaN,다둥이
14557,13894,F,30세~34세,460,NaN,롭스
14558,13894,F,30세~34세,460,NaN,더영
14559,13894,F,30세~34세,460,NaN,하이마트


In [132]:
dfc=df
dfc=dfc.drop_duplicates(['고객번호'])
dfc

,고객번호,성별,연령대,거주지역,경쟁사이용횟수,멤버쉽명
0,00001,M,60세이상,060,15.0,NaN
1,00002,M,60세이상,100,8.0,NaN
2,00003,M,60세이상,033,NaN,NaN
3,00004,F,60세이상,016,3.0,NaN
4,00005,M,60세이상,100,NaN,NaN
...,...,...,...,...,...,...
20336,19379,F,20세~24세,069,NaN,롭스
20337,19380,F,20세~24세,039,NaN,롭스
20338,19381,F,40세~44세,460,NaN,다둥이
20339,19382,F,45세~49세,460,NaN,NaN


In [145]:
%%time
for i,j in zip(df['고객번호'],df['멤버쉽명']):
    a=df[df['고객번호'] == i]['멤버쉽명'].tolist()
    
    if len(a) >= 2:
        dfc.loc[dfc['고객번호'] == i,'멤버쉽명']=",".join(a)

Wall time: 48.2 s


In [147]:
dfc[dfc['고객번호'] == '13894']

,고객번호,성별,연령대,거주지역,경쟁사이용횟수,멤버쉽명
14556,13894,F,30세~34세,460,NaN,"다둥이,롭스,더영,하이마트"
